In [1]:
# Librerías.
import pandas as pd
import Funciones as f
import unidecode
from docx import Document
from docx.shared import Inches
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scikit_posthocs as sp
import openpyxl
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font

In [2]:
%%capture
%run "25. Análisis descriptivo de ítems de CO individuales.ipynb"

In [3]:
def Extraer_Numero_Item(Nombre_Columna: str) -> int:

    """
    
    Extrae el número de ítem desde el nombre de columna.
    
    Ejemplo:
    >>> Extraer_Numero_Item("CO_Item_9_Izq")
    9
    
    """

    return int(Nombre_Columna.split('_')[2])

In [4]:
def Determinar_Escala_Numero(Numero_Item: int) -> str:

    """
    
    Determina si el ítem pertenece a progresismo o conservadurismo.
    
    Ejemplo:
    >>> Determinar_Escala_Numero(5)
    "PROGRESISMO"
    
    """

    if Numero_Item in Items_Progresistas:
        return "PROGRESISMO"
    elif Numero_Item in Items_Conservadores:
        return "CONSERVADURISMO"
    return "DESCONOCIDA"

In [5]:
def Determinar_Lado_Item(Nombre_Columna: str) -> str:

    """
    
    Clasifica el candidato como Izquierda o Derecha por sufijo.
    
    Ejemplo:
    >>> Determinar_Lado_Item("CO_Item_4_Der")
    "Derecha"
    
    """

    if Nombre_Columna.endswith("Izq"):
        return "Izquierda"
    return "Derecha"

In [6]:
def Determinar_Tipo_Item(Nombre_Columna: str) -> str:

    """
    
    Determina si el ítem es de Opinión o de Tiempo por prefijo.
    
    Ejemplo:
    >>> Determinar_Tipo_Item("CT_Item_3_Izq")
    "TIEMPO"
    
    """

    if Nombre_Columna.startswith("CO_"):
        return "OPINION"
    elif Nombre_Columna.startswith("CT_"):
        return "TIEMPO"
    return "DESCONOCIDO"

In [7]:
# Ordenar las columnas de interés por número de ítem.
Columnas_Ordenadas = sorted(
    Columnas_Cambios,
    key = lambda Nombre: Extraer_Numero_Item(Nombre)
)

In [9]:
for Nombre_df, df in dfs_Finales.items():

    # Obtener las categorías únicas ordenadas.
    Categorias_Ordenadas = list(Diccionario_Resultados_CO_Individuales[Nombre_df].keys())
    Categorias_Ordenadas.sort()

    # Crear el libro y la hoja de trabajo de Excel.
    Libro_Excel = Workbook()
    Hoja_Trabajo = Libro_Excel.active
    Hoja_Trabajo.title = f"Resumen_Estadistico_{Nombre_df}"

    # Fila actual en la hoja de trabajo.
    Fila_Actual = 1

    # Iterar sobre cada columna de interés.
    for Nombre_Columna in Columnas_Ordenadas:
        Numero_Item = Extraer_Numero_Item(Nombre_Columna)
        Escala = Determinar_Escala_Numero(Numero_Item)
        Lado = Determinar_Lado_Item(Nombre_Columna)
        Tipo_Item = Determinar_Tipo_Item(Nombre_Columna)

        # Encabezado de bloque.
        Hoja_Trabajo.cell(row = Fila_Actual, column = 1,
            value = "ÍTEM").font = Font(bold = True)
        Hoja_Trabajo.cell(row = Fila_Actual, column = 2,
            value = "ESCALA").font = Font(bold = True)
        Hoja_Trabajo.cell(row = Fila_Actual, column = 3,
            value = "CANDIDATO").font = Font(bold = True)
        Hoja_Trabajo.cell(row = Fila_Actual, column = 4,
            value = "TIPO_ITEM").font = Font(bold = True)
        Hoja_Trabajo.cell(row = Fila_Actual, column = 5,
            value = "MEDIA").font = Font(bold = True)
        Hoja_Trabajo.cell(row = Fila_Actual, column = 6,
            value = "MEDIANA").font = Font(bold = True)
        Hoja_Trabajo.cell(row = Fila_Actual, column = 7,
            value = "ES").font = Font(bold = True)
        Hoja_Trabajo.cell(row = Fila_Actual, column = 8,
            value = "N").font = Font(bold = True)

        Fila_Actual += 1

        # Metadatos de ítem en una sola fila.
        Hoja_Trabajo.cell(row = Fila_Actual, column = 1,
            value = Numero_Item)
        Hoja_Trabajo.cell(row = Fila_Actual, column = 2,
            value = Escala)
        Hoja_Trabajo.cell(row = Fila_Actual, column = 3,
            value = Lado)
        Hoja_Trabajo.cell(row = Fila_Actual, column = 4,
            value = Tipo_Item)

        Fila_Actual += 1

        # Estadísticas por categoría.
        for Categoria in Categorias_Ordenadas:
            Estadisticas = Diccionario_Resultados_CO_Individuales[Nombre_df][Categoria]\
                .get(Nombre_Columna, {})
            Media = Estadisticas.get("Media", "")
            Mediana = Estadisticas.get("Mediana", "")
            Error_Estandar = Estadisticas.get("Error_Estandar", "")
            N = Estadisticas.get("N", "")

            Hoja_Trabajo.cell(row = Fila_Actual, column = 3,
                value = Categoria)
            Hoja_Trabajo.cell(row = Fila_Actual, column = 5,
                value = Media)
            Hoja_Trabajo.cell(row = Fila_Actual, column = 6,
                value = Mediana)
            Hoja_Trabajo.cell(row = Fila_Actual, column = 7,
                value = Error_Estandar)
            Hoja_Trabajo.cell(row = Fila_Actual, column = 8,
                value = N)

            Fila_Actual += 1

        # Fila blanca antes del siguiente ítem.
        Fila_Actual += 1
    
    # Guardar el archivo Excel.
    Libro_Excel.save(f"Resumen_Estadistico_Final_{Nombre_df}.xlsx")